In [ ]:
!pip install transformers datasets torch scikit-learn


In [ ]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
)
from datasets import load_dataset, Dataset
from sklearn.model_selection import train_test_split

# KoELECTRA Tokenizer 및 Model 로드
model_name = "beomi/koelectra-base-v3-discriminator"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=5
)  # 치과 관련 카테고리 개수만큼 설정

# 치과 관련 데이터셋 준비
# 여기에 치과 관련 텍스트와 라벨이 있어야 합니다.
# 예시 데이터 (질문과 라벨)
texts = [
    "어금니가 너무 아파요, 신경치료가 필요할까요?",
    "잇몸에서 피가 나요, 잇몸 치료가 필요해요.",
    "앞니를 때웠는데, 보철물 문제가 있는 것 같아요.",
    "치아가 시려요, 충치 때문인가요?",
    "교정 치료가 필요한가요? 치아가 삐뚤어요.",
]

labels = [
    0,
    1,
    2,
    0,
    3,
]  # 각 라벨에 해당하는 진료 항목 (0: 신경치료, 1: 잇몸 치료, 2: 보철물 문제, 3: 교정 치료)

# 데이터셋 생성
train_texts, val_texts, train_labels, val_labels = train_test_split(
    texts, labels, test_size=0.2
)

train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)


# Pytorch Dataset 클래스로 변환
class DentalDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


train_dataset = DentalDataset(train_encodings, train_labels)
val_dataset = DentalDataset(val_encodings, val_labels)

# TrainingArguments 설정
training_args = TrainingArguments(
    output_dir="./results",  # 학습 결과 저장 경로
    num_train_epochs=3,  # 학습 에포크 수
    per_device_train_batch_size=8,  # 배치 크기
    per_device_eval_batch_size=8,  # 평가 시 배치 크기
    warmup_steps=500,  # 학습 시작 전에 learning rate가 증가하는 단계 수
    weight_decay=0.01,  # 가중치 감소 (regularization)
    logging_dir="./logs",  # 로그 저장 경로
    logging_steps=10,  # 로그 남길 스텝 간격
    evaluation_strategy="steps",  # 평가 전략 (에포크별 또는 스텝별 평가)
    eval_steps=50,  # 평가할 스텝 간격
    save_steps=100,  # 모델을 저장할 스텝 간격
    load_best_model_at_end=True,  # 최고의 성능을 보인 모델을 마지막에 로드
)

# Trainer 설정
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# 모델 학습
trainer.train()

# 모델 평가
trainer.evaluate()

# 모델 저장
trainer.save_model("./fine_tuned_koelectra_dental")
tokenizer.save_pretrained("./fine_tuned_koelectra_dental")